# ESTUDIO DE FACTORES DE MOTIVACIÓN EN LA COMPRA DE UN VEHÍCULO ELÉCTRICO Y LA IMPLANTACIÓN MASIVA DE ESTOS ENTRE LOS CONDUCTORES
## ELENA DELGADO DEL REY 
## TRABAJO FIN DE GRADO

Este trabajo va sobre los coches eléctricos que se encuentran actualmente en el mercado. En el DataSet utilizado tenemos información de la marca del coche, su modelo, precio, etc...

# PREPARAR EL ENTORNO

In [393]:
import pandas as pd 
import numpy as np      
!pip install -q pyspark
%matplotlib inline
!pip install -q findspark
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()
import pyspark
from pyspark.sql.session import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as func
from pyspark.sql.functions import col
import pyspark.sql.types as typ
import pyspark.ml.feature as ft
import pyspark.ml.regression as rg
from pyspark.ml import Pipeline
import pyspark.ml.evaluation as ev
from pyspark.ml.param import Param, Params
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import GBTRegressor
import pyspark.ml.regression
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.feature import VectorAssembler, VectorIndexer
from pyspark.ml.regression import LinearRegression
import pyspark.ml.tuning as tune
import numpy as np
from pyspark.mllib.linalg import Vectors
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import PCA
from pyspark.ml.feature import ChiSqSelector

In [394]:
import os
exec(open(os.path.join(os.environ["SPARK_HOME"], 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 3.3.0
      /_/

Using Python version 3.7.13 (default, Apr 24 2022 01:04:09)
Spark context Web UI available at http://fb5eb0219f58:4040
Spark context available as 'sc' (master = local[*], app id = local-1656521760155).
SparkSession available as 'spark'.


In [395]:
import matplotlib.pyplot as plt
import seaborn as sns
import statistics as stats
%matplotlib inline 
sns.set_palette('pastel')
import matplotlib.pyplot as plt
from matplotlib import colors as mcolors
import pandas.util.testing as tm
from google.colab import files

# IMPORTAMOS EL CSV QUE SE HA LIMPIADO Y UNIFICADO PREVIAMENTE

In [396]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
# para utilizar lo que tengo en google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [397]:
spark = SparkSession.builder.master("local[*]").getOrCreate()

Primero vamos a importar el csv, indicandole que los datos estan separados con  ; y que la primera columna es el header del dataframe

In [398]:
coches_df = spark.read.csv("/content/drive/MyDrive/Cuarto/TFG/Coches/DatasetVE.csv", header='true', inferSchema='false', schema=schema, sep=';')

In [399]:
coches_df.head(1)

[Row(Marca0=1, MARCA1='VOLKSWAGEN', MODELO='ID. BUZZ', Ventas total=6323, Precio=55000, range km=340, tiempo=10.0, kWh=77.0, eficiencia Wh/km=226, cargarapida km/h=510, velocidadmax km/h=145, Segment_num=7, marketsegment='N', personas=4, towingcapacity=1134, motor='Battery Electric Vehicle ', url='https://ev-database.org/car/1651/1-ID-Buzz')]

Vamos a visualizar el esquema, para entender de que tipo sn las variables

In [400]:
coches_df.printSchema()

root
 |-- Marca0: integer (nullable = true)
 |-- MARCA1: string (nullable = true)
 |-- MODELO: string (nullable = true)
 |-- Ventas total: integer (nullable = true)
 |-- Precio: integer (nullable = true)
 |-- range km: integer (nullable = true)
 |-- tiempo: double (nullable = true)
 |-- kWh: double (nullable = true)
 |-- eficiencia Wh/km: integer (nullable = true)
 |-- cargarapida km/h: integer (nullable = true)
 |-- velocidadmax km/h: integer (nullable = true)
 |-- Segment_num: integer (nullable = true)
 |-- marketsegment: string (nullable = true)
 |-- personas: integer (nullable = true)
 |-- towingcapacity: integer (nullable = true)
 |-- motor: string (nullable = true)
 |-- url: string (nullable = true)



A continuación, vamos a crear un nuevo dataframe indicandole únicamente las variables que vamos a querer utilizar en los modelos. Teniendo en cuenta que al ser regressores las variables tienen que ser númericas

In [401]:
cochesnuevo=coches_df.select("Marca0","Ventas total","Precio","range km","tiempo","kWh","eficiencia Wh/km","cargarapida km/h","velocidadmax km/h","Segment_num","personas")

In [402]:
cochesnuevo.show()

+------+------------+------+--------+------+----+----------------+----------------+-----------------+-----------+--------+
|Marca0|Ventas total|Precio|range km|tiempo| kWh|eficiencia Wh/km|cargarapida km/h|velocidadmax km/h|Segment_num|personas|
+------+------------+------+--------+------+----+----------------+----------------+-----------------+-----------+--------+
|     1|        6323| 55000|     340|  10.0|77.0|             226|             510|              145|          7|       4|
|     1|        6323| 47740|     410|   8.5|77.0|             188|             520|              160|          3|       5|
|     1|        6323| 36890|     350|   9.6|58.0|             166|             490|              160|          3|       4|
|     1|        6323| 53790|     400|   6.2|77.0|             193|             500|              180|          3|       5|
|     1|        6323| 26895|     205|  11.9|32.3|             158|             170|              130|          1|       4|
|     1|        

Una vez tenemos todos los Datasets revisados, vamos a entrenar nuestro modelo. Como nuestro Dataset no fue sacado de los challenges y nuestro proposito desde que empezamos a manejar los datos fue predecir el precio futuro de un coche, ahora vamos a estudiar con métodos de predicción adecuados los precios.

# MODELOS DE APRENDIZAJE

In [403]:
import pyspark.ml.feature as ft
import pyspark.ml.regression as rg
from pyspark.ml import Pipeline
import pyspark.ml.evaluation as ev
from pyspark.ml.feature import VectorAssembler, VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.param import Param, Params

Un modelo de regresión es un modelo matemático que busca determinar la relación entre una variable dependiente (Y) con respecto a otras variables llamadas explicativas o independientes (X). Asimismo, el modelo busca determinar cuál será el impacto sobre la variable Y ante un cambio en las variables explicativas (X).

Puesto que nosotros tenemos que hacer una regresión, vamos a ver que métodos de entrenamiento tenemos. 
+ Linear regression
+ Decision tree regression
+ Random forest regression
+ Gradient-boosted tree regression
+ Isotonic regression

## DIVISIÓN DE DATOS

Vamos a dividir los datos, 80 % va a ser de los datos de entrenamiento y el otro 20 % es para los datos de test. Ambos que luego se usaran para entrenar el modelo

In [405]:
# Split the data into training and test sets (20% held out for testing)
(trainingData, testData) = cochesnuevo.randomSplit([0.8, 0.2])

Antes de comenzar, vamos a pasar nuestra columna de price para que sea de DoubleType y así podamos usarla en los modelos.

In [406]:
cochesnuevo = cochesnuevo.withColumn('Precio', cochesnuevo['Precio'].cast(typ.DoubleType()))

## PASOS A SEGUIR PARA CREAR PREDICCIONES 

1. Para realizar este algoritmo vamos a empezar creando una columna que reúna todas las variables utilizando los **transformadores**.
2. Preparar nuestro **modelo** con sus respectivos valores en caso de tener.
3. Creamos el **pipeline**. El Pipeline nos va a ayudar a meter nuestros transformadores y el modelo para ejecutarlo uno detrás de otro.
4. **Modelamos** nuestros datos de training el cual tiene el 80% de nuestros datos.
5. A continuación **testeamos** el modelo 
6. **Visualizamos** las predicciones que hemos obtenido.
7. **Evaluamos** el modelo 
8. Calculamos el **RMSE** (raíz del error cuadrático medio) y el **MSE** (error cuadrático medio)  de nuestro modelo. 

## **RANDOM FOREST**

In [407]:
from pyspark.ml.regression import RandomForestRegressor

El primer algoritmo ES Random Forest ya que este algoritmo de regresión consiste en un conjunto de múltiples árboles de decisión independientes, al cual se le asigna un conjunto de datos aleatorios con una misma distribución. La salida es el promedio de los resultados finales de cada árbol.

Creamos el estimador en nuestro caso es un modelo de tipo Regresión y escogemos el Random Forest

Random forest:es un método en nuestro caso para regresión, contruyendo multiplos árboles de decision y te crea una multiplicación.

Vamos a empezar creando una columna que reúna todas las variables utilizando el transformador VectorAssembler.

In [408]:
mivectorRF = ft.VectorAssembler(
    inputCols=["Marca0","Ventas total","range km","tiempo","kWh","eficiencia Wh/km","cargarapida km/h","velocidadmax km/h","Segment_num","personas"], 
    outputCol='features'
)

+ Ahora vamos a preparar nuestro modelo con sus valores:
    + 5 árboles para entrenar nuestros datos
    + 5 niveles de profundidad de los árboles
    + 'price', la columna que tiene que predecir

In [409]:
algoritmoRF = rg.RandomForestRegressor(
    numTrees=5, 
    maxDepth=5, 
    labelCol='Precio')

Utilizamos el Pipeline ya que nos va a ayudar a meter nuestro VectorAssembler y el modelo para ejecutarlo uno detrás de otro.

In [427]:
pipelineRF = Pipeline(stages=[mivectorRF, algoritmoRF])

Modelamos nuestros datos de training el cual tiene el 80% de nuestros datos.

In [428]:
modelRF = pipelineRF.fit(trainingData)

Ahora vamos a poner al modelo a que haga las prediciones con los datos.

In [429]:
testRF = modelRF.transform(testData)

Visualizamos las dos primeras columnas para ver que prediccion nos ha hecho nuestro modelo.

In [430]:
testRF.take(2)

[Row(Marca0=1, Ventas total=6323, Precio=42460.0, range km=450, tiempo=7.9, kWh=77.0, eficiencia Wh/km=171, cargarapida km/h=570, velocidadmax km/h=160, Segment_num=3, personas=4, features=DenseVector([1.0, 6323.0, 450.0, 7.9, 77.0, 171.0, 570.0, 160.0, 3.0, 4.0]), prediction=44951.26741234425),
 Row(Marca0=1, Ventas total=6323, Precio=43807.0, range km=410, tiempo=10.4, kWh=77.0, eficiencia Wh/km=188, cargarapida km/h=520, velocidadmax km/h=160, Segment_num=3, personas=5, features=DenseVector([1.0, 6323.0, 410.0, 10.4, 77.0, 188.0, 520.0, 160.0, 3.0, 5.0]), prediction=53889.599204126745)]

Ahora vamos a evaluar nuestro modelo para ver la raíz del error cuadratico medio RMSE, y el error cuadrático medio MSE.

In [459]:
evaluator = RegressionEvaluator(
    labelCol="Precio", predictionCol="prediction", metricName="rmse")
rmseRF = evaluator.evaluate(testRF)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmseRF) 

Root Mean Squared Error (RMSE) on test data = 32453.5


In [460]:
mseRF = rmseRF*rmseRF
print("Mean Squared Error (MSE) on test data = %g" % mseRF) 

Mean Squared Error (MSE) on test data = 1.05323e+09


## **GRADIENT BOOSTED TREE CLASSIFIER**

In [419]:
from pyspark.ml.regression import GBTRegressor

Gradient Boosted Tree Regressor es una técnica de aprendizaje automático que produce un modelo de predicción en forma de un conjunto de modelos de predicción débiles, típicamente árboles de decisión. En estos casos, construye cada árbol de regresión de forma gradual, utilizando una función de pérdida predefinida para medir el error en cada paso y corregirlo en el siguiente.

Habiendo visto que Random Forest no es un modelo adecuado para nuestros datos, se va a comprobar si este modelo nos proporciona mejores predicciones.



Creamos el estimador en nuestro caso es un modelo de tipo Regresión y escogemos el Gradient Boosted Tree Classifier

Gracient Boosted Tree Classifier: es un grupo de algoritmos que combina métodos juntos para crear un modelo de predicción. 

Al igual que con el método anterior, vamos a empezar creando una columna que reúna todas las variables utilizando transformadores.

In [435]:
featuresCols = cochesnuevo.columns

In [422]:
mivectorGBT = VectorAssembler(inputCols=featuresCols, outputCol="rawFeatures")

In [436]:
# This identifies categorical features and indexes them.
vectorIndexer = VectorIndexer(inputCol="rawFeatures", outputCol="features", maxCategories=4)

Preparamos el modelo con columna a predecir 'price'

In [437]:
# Takes the "features" column and learns to predict "hotel_cluster"
gbt = GBTRegressor(labelCol="Precio")

Creamos el pipeline para ejecutar nuestro VectorAssembler, el VectorIndexer y el modelo uno detrás de otro.

In [438]:
pipelineGBT = Pipeline(stages=[mivectorGBT, vectorIndexer, gbt])

Realizamos el modelado de nuestros datos de training (80% de los datos).

In [444]:
modeloGBT = pipelineGBT.fit(trainingData)

Entrenamos el modelo con los datos de test (20%)

In [445]:
testGBT = modeloGBT.transform(testData)

Visualizamos las dos primeras prediciones para ver como ha funcionado nuestro modelo

In [446]:
testGBT.take(2)

[Row(Marca0=1, Ventas total=6323, Precio=42460.0, range km=450, tiempo=7.9, kWh=77.0, eficiencia Wh/km=171, cargarapida km/h=570, velocidadmax km/h=160, Segment_num=3, personas=4, rawFeatures=DenseVector([1.0, 6323.0, 42460.0, 450.0, 7.9, 77.0, 171.0, 570.0, 160.0, 3.0, 4.0]), features=DenseVector([1.0, 6323.0, 42460.0, 450.0, 7.9, 77.0, 171.0, 570.0, 160.0, 3.0, 4.0]), prediction=43201.14237522483),
 Row(Marca0=1, Ventas total=6323, Precio=43807.0, range km=410, tiempo=10.4, kWh=77.0, eficiencia Wh/km=188, cargarapida km/h=520, velocidadmax km/h=160, Segment_num=3, personas=5, rawFeatures=DenseVector([1.0, 6323.0, 43807.0, 410.0, 10.4, 77.0, 188.0, 520.0, 160.0, 3.0, 5.0]), features=DenseVector([1.0, 6323.0, 43807.0, 410.0, 10.4, 77.0, 188.0, 520.0, 160.0, 3.0, 5.0]), prediction=44369.604280154104)]

Evaluamos la precisión de predicción de nuestro modelo con un RegressionEvaluator

In [447]:
evaluator = RegressionEvaluator(
    labelCol="Precio", predictionCol="prediction", metricName="rmse")

Calculamos el RMSE (raíz del error cuadrático medio) y el MSE (error cuadrático medio) de nuestro modelo.

In [461]:
rmseGBT = evaluator.evaluate(testGBT)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmseGBT)

Root Mean Squared Error (RMSE) on test data = 5520.53


In [449]:
mseGBT = rmseGBT*rmseGBT
print("Mean Squared Error (MSE) on test data = %g" % mseGBT) 

Mean Squared Error (MSE) on test data = 3.04763e+07


## **LINEAR REGRESSION**

In [450]:
from pyspark.ml.regression import LinearRegression

Linear regression es una técnica estadística donde podemos identificar que variables independientes (causas) explican una variable dependiente (resultado).

Creamos el estimador en nuestro caso es un modelo de tipo Regresión y escogemos el Linear Regression

Linear regression: es un enfoque linear para modelar la relación entre dos escalares (variables dependientes) y una o mas varibles independientes

Comenzamos creando una columna que reúna todas las variables utilizando el transformador VectorAssembler. A esta columna la llamaremos 'rawFeatures'

In [451]:
mivectorLR = VectorAssembler(inputCols=featuresCols, outputCol="rawFeatures")

In [452]:
# This identifies categorical features and indexes them.
vectorIndexerLR = VectorIndexer(inputCol="rawFeatures", outputCol="features", maxCategories=4)

Montamos el modelo de Linear Regresision para predecir la columna 'price'

In [360]:
# Takes the "features" column and learns to predict "price_cluster"
lr = LinearRegression(labelCol="Precio")

Al igual que los casos anteriores, creamos el pipeline para ejecutar nuestro VectorAssembler, el VectorIndexer y el modelo uno detrás de otro.

In [453]:
pipelineLR = Pipeline(stages=[mivectorLR, vectorIndexerLR, lr])

Realizamos el modelado del 80% de los datos y entrenamos el modelo con el 20% restante

In [454]:
modelLR = pipelineLR.fit(trainingData)

In [455]:
testLR = modelLR.transform(testData)

Visualizamos las dos primeras prediciones para ver como ha funcionado nuestro modelo

In [457]:
testLR.take(2)

[Row(Marca0=1, Ventas total=6323, Precio=42460.0, range km=450, tiempo=7.9, kWh=77.0, eficiencia Wh/km=171, cargarapida km/h=570, velocidadmax km/h=160, Segment_num=3, personas=4, rawFeatures=DenseVector([1.0, 6323.0, 42460.0, 450.0, 7.9, 77.0, 171.0, 570.0, 160.0, 3.0, 4.0]), features=DenseVector([1.0, 6323.0, 42460.0, 450.0, 7.9, 77.0, 171.0, 570.0, 160.0, 3.0, 4.0]), prediction=42459.67155366662),
 Row(Marca0=1, Ventas total=6323, Precio=43807.0, range km=410, tiempo=10.4, kWh=77.0, eficiencia Wh/km=188, cargarapida km/h=520, velocidadmax km/h=160, Segment_num=3, personas=5, rawFeatures=DenseVector([1.0, 6323.0, 43807.0, 410.0, 10.4, 77.0, 188.0, 520.0, 160.0, 3.0, 5.0]), features=DenseVector([1.0, 6323.0, 43807.0, 410.0, 10.4, 77.0, 188.0, 520.0, 160.0, 3.0, 5.0]), prediction=43807.47859376746)]

Evaluamos la precisión de predicción de nuestro modelo con un RegressionEvaluator. Al ver que las predicciones de linear regression son bastante parecidas a las originales podemos suponer que el error cuadrático medio será muy bajo.

In [458]:
# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="Precio", predictionCol="prediction", metricName="rmse")
rmseLR = evaluator.evaluate(testLR)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmseLR)

Root Mean Squared Error (RMSE) on test data = 5.27958


Calculamos el MSE (error cuadrático medio) de nuestro modelo.

In [462]:
mseLR = rmseLR*rmseLR
print("Mean Squared Error (MSE) on test data = %g" % mseLR) 

Mean Squared Error (MSE) on test data = 27.8739


## **GRID SEARCH de random forest**

Vamos a hacer un grid search de random forest para ver que parametros son los mejores para la prediccion de datos.Grid search es el proceso de escanear los datos para configurar parámetros óptimos para un modelo dado. Grid-Search creará un modelo en cada combinación de parámetros posible. Recorre cada combinación de parámetros y almacena un modelo para cada combinación.

In [463]:
import pyspark.ml.regression as rg
from pyspark.ml.regression import RandomForestRegressor

forest =  rg.RandomForestRegressor(labelCol='Precio')

grid = tune.ParamGridBuilder().addGrid(forest.numTrees, [2, 5]).addGrid(forest.maxDepth, [5, 3]).build()

In [464]:
evaluator =RegressionEvaluator(predictionCol='prediction', labelCol='Precio')

In [466]:
rgC = tune.CrossValidator(estimator=forest, estimatorParamMaps=grid, evaluator=evaluator)

In [467]:
pipeline = Pipeline(stages=[mivectorRF])
data_transformer = pipeline.fit(trainingData)

In [468]:
data_transformer = pipeline.fit(trainingData)

In [469]:
rgModel = rgC.fit(data_transformer.transform(trainingData))

In [470]:
data_train = data_transformer.transform(testData)

In [471]:
results = rgModel.transform(data_train)

In [472]:
results.take(2)

[Row(Marca0=1, Ventas total=6323, Precio=42460.0, range km=450, tiempo=7.9, kWh=77.0, eficiencia Wh/km=171, cargarapida km/h=570, velocidadmax km/h=160, Segment_num=3, personas=4, features=DenseVector([1.0, 6323.0, 450.0, 7.9, 77.0, 171.0, 570.0, 160.0, 3.0, 4.0]), prediction=46604.3789890374),
 Row(Marca0=1, Ventas total=6323, Precio=43807.0, range km=410, tiempo=10.4, kWh=77.0, eficiencia Wh/km=188, cargarapida km/h=520, velocidadmax km/h=160, Segment_num=3, personas=5, features=DenseVector([1.0, 6323.0, 410.0, 10.4, 77.0, 188.0, 520.0, 160.0, 3.0, 5.0]), prediction=53406.180373087875)]

In [473]:
# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="Precio", predictionCol="prediction", metricName="rmse")
rmseGS = evaluator.evaluate(results)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmseGS)

Root Mean Squared Error (RMSE) on test data = 24814.5


In [474]:
mseGS = rmseGS*rmseGS
print("Mean Squared Error (MSE) on test data = %g" % mseGS) 

Mean Squared Error (MSE) on test data = 6.15758e+08


Podemos observar que, aunque tenga los parametros más adecuados, el error de predicción sigue siendo muy alto, por lo que esto nos confirma que Random Forest NO es el método adecuado para nuestros datos

In [475]:
results = [
    (
        [
            {key.name: paramValue} 
            for key, paramValue 
            in zip(
                params.keys(), 
                params.values())
        ], metric
    ) 
    for params, metric 
    in zip(
        rgModel.getEstimatorParamMaps(), 
        rgModel.avgMetrics
    )
]

sorted(results, key=lambda el: el[1], reverse=True)[0]

([{'numTrees': 2}, {'maxDepth': 5}], 31970.50331774612)

Como podemos ver el RMSE ha bajado para el Random Forest

## DISCRETIZAR VARIABLES CONTINUAS

Discretizar datos quiere decir convertir variables que son continuas en variables agrupadas por intervalos. Esta operación simplifica la información agrupando los objetos geográficos que presentan las mismas características en distintas clases. Hemos discretizado la columna de Price, ya que esta al ser una medición es una variable contínua.

In [476]:
x = np.arange(0, 100)
x = x / 100.0 * np.pi * 4
y = x * np.sin(x / 1.764) + 20.1234

schema = typ.StructType([
    typ.StructField('Precio', 
                    typ.DoubleType(), 
                    False
   )
])

data = sqlContext.createDataFrame([[float(e), ] for e in y], schema=schema)
#data = spark.createDataFrame([[float(e), ] for e in y], schema=schema)

Vamos a utilizar el QuantileDiscretizer para partir nuestra variable continua en 5 buckets

In [477]:
discretizer = ft.QuantileDiscretizer(
    numBuckets=5, 
    inputCol='Precio', 
    outputCol='discretized')

In [479]:
data_discretized = discretizer.fit(cochesnuevo).transform(cochesnuevo)

data_discretized \
    .groupby('discretized')\
    .mean('Precio')\
    .sort('discretized')\
    .collect()

[Row(discretized=0.0, avg(Precio)=30761.204545454544),
 Row(discretized=1.0, avg(Precio)=43216.704545454544),
 Row(discretized=2.0, avg(Precio)=53590.568181818184),
 Row(discretized=3.0, avg(Precio)=63489.545454545456),
 Row(discretized=4.0, avg(Precio)=114160.51111111112)]

## **LINEAR REGRESSION DISCRETIZED**

Como Regresión Lineal es el algoritmo que mejor predicción nos ha dado, vamos a utilizarlo con los datos discretizados que acabamos de hallar.

Al tener nuevos datos, debemos volver a separar nuestros datos en entrenamiento y test en un 80-20

In [480]:
(trainingDataD, testDataD) = data_discretized.randomSplit([0.8, 0.2])

Realizamos los MISMOS PASOS que Linear Regressionn

In [481]:
modelLRD = pipelineLR.fit(trainingDataD)

In [482]:
testLRD = modelLRD.transform(testDataD)

In [483]:
testLRD.take(2)

[Row(Marca0=1, Ventas total=6323, Precio=38390.0, range km=350, tiempo=7.3, kWh=58.0, eficiencia Wh/km=166, cargarapida km/h=490, velocidadmax km/h=160, Segment_num=3, personas=4, discretized=0.0, rawFeatures=DenseVector([1.0, 6323.0, 38390.0, 350.0, 7.3, 58.0, 166.0, 490.0, 160.0, 3.0, 4.0]), features=DenseVector([1.0, 6323.0, 38390.0, 350.0, 7.3, 58.0, 166.0, 490.0, 160.0, 3.0, 4.0]), prediction=38391.17197737509),
 Row(Marca0=1, Ventas total=6323, Precio=55690.0, range km=405, tiempo=6.3, kWh=77.0, eficiencia Wh/km=190, cargarapida km/h=540, velocidadmax km/h=180, Segment_num=3, personas=5, discretized=2.0, rawFeatures=DenseVector([1.0, 6323.0, 55690.0, 405.0, 6.3, 77.0, 190.0, 540.0, 180.0, 3.0, 5.0]), features=DenseVector([1.0, 6323.0, 55690.0, 405.0, 6.3, 77.0, 190.0, 540.0, 180.0, 3.0, 5.0]), prediction=55689.331184753806)]

In [485]:
# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="Precio", predictionCol="prediction", metricName="rmse")
rmseLRD = evaluator.evaluate(testLRD)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmseLRD)

Root Mean Squared Error (RMSE) on test data = 5.86492


In [486]:
mseLRD = rmseLRD*rmseLRD
print("Mean Squared Error (MSE) on test data = %g" % mseLRD) 

Mean Squared Error (MSE) on test data = 34.3973


## ESTANDARIZAR VARIABLES CONTINUAS

La estandarización o normalización de índices significa ajustar los valores medidos en diferentes escalas respecto a una escala común, a menudo previo a un proceso de realizar promedios.

In [493]:
vectorizer = ft.VectorAssembler(inputCols=['Precio'], outputCol= 'Precio_vec')

Construimos el "normalizer" y el "pipeline". Pondremos "withMean=True" y "withStd=True" para que la media y la varianza sean de un longitud de uno.

In [494]:
normalizer = ft.StandardScaler(
    inputCol=vectorizer.getOutputCol(), 
    outputCol='normalized', 
    withMean=True,
    withStd=True
)

In [495]:
pipeline = Pipeline(stages=[vectorizer, normalizer])
data_standardized = pipeline.fit(cochesnuevo).transform(cochesnuevo)

In [496]:
data_standardized.take(1)

[Row(Marca0=1, Ventas total=6323, Precio=55000.0, range km=340, tiempo=10.0, kWh=77.0, eficiencia Wh/km=226, cargarapida km/h=510, velocidadmax km/h=145, Segment_num=7, personas=4, Precio_vec=DenseVector([55000.0]), normalized=DenseVector([-0.1884]))]

## **LINEAR REGRESSIÓN STANDARIZED**

Al igual que en el caso anterior, vamos a utilizar Regresión Lineal ya que es el algoritmo que mejor predicción nos ha dado, por lo vamos a utilizarlo con los datos estandarizados que acabamos de hallar.

In [497]:
(trainingDataE, testDataE) = data_discretized.randomSplit([0.8, 0.2])

Realizamos los MISMOS PASOS que Linear Regressionn

In [498]:
modelLRE = pipelineLR.fit(trainingDataE)

In [499]:
testLRE = modelLRE.transform(testDataE)

In [500]:
testLRE.take(2)

[Row(Marca0=1, Ventas total=6323, Precio=33990.0, range km=275, tiempo=8.9, kWh=45.0, eficiencia Wh/km=164, cargarapida km/h=410, velocidadmax km/h=160, Segment_num=3, personas=4, discretized=0.0, rawFeatures=DenseVector([1.0, 6323.0, 33990.0, 275.0, 8.9, 45.0, 164.0, 410.0, 160.0, 3.0, 4.0]), features=DenseVector([1.0, 6323.0, 33990.0, 275.0, 8.9, 45.0, 164.0, 410.0, 160.0, 3.0, 4.0]), prediction=33991.585923871506),
 Row(Marca0=1, Ventas total=6323, Precio=36890.0, range km=350, tiempo=9.6, kWh=58.0, eficiencia Wh/km=166, cargarapida km/h=490, velocidadmax km/h=160, Segment_num=3, personas=4, discretized=0.0, rawFeatures=DenseVector([1.0, 6323.0, 36890.0, 350.0, 9.6, 58.0, 166.0, 490.0, 160.0, 3.0, 4.0]), features=DenseVector([1.0, 6323.0, 36890.0, 350.0, 9.6, 58.0, 166.0, 490.0, 160.0, 3.0, 4.0]), prediction=36891.3268566916)]

In [501]:
# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="Precio", predictionCol="prediction", metricName="rmse")
rmseLRE = evaluator.evaluate(testLRE)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmseLRE)

Root Mean Squared Error (RMSE) on test data = 4.33176


In [502]:
mseLRE = rmseLRE*rmseLRE
print("Mean Squared Error (MSE) on test data = %g" % mseLRE) 

Mean Squared Error (MSE) on test data = 18.7641


# **COMPARACIÓN RESULTADOS**

In [511]:
print("RANDOM FOREST: MSE = %g" % mseRF ) 
print("RANDOM FOREST + GRID SEARCH: MSE = %g" % mseGS )
print("GRADIENT BOOSTED TREE REGRESSOR: MSE = %g" % mseGBT )
print("LINEAR REGRESSION + DISCRETIZADOS:MSE = %g" % mseLRD ) 
print("LINEAR REGRESSION: MSE = %g" % mseLR )
print("LINEAR REGRESSION + ESTANDARIZADOS: MSE = %g" % mseLRE ) 

RANDOM FOREST: MSE = 1.05323e+09
RANDOM FOREST + GRID SEARCH: MSE = 6.15758e+08
GRADIENT BOOSTED TREE REGRESSOR: MSE = 3.04763e+07
LINEAR REGRESSION + DISCRETIZADOS:MSE = 34.3973
LINEAR REGRESSION: MSE = 27.8739
LINEAR REGRESSION + ESTANDARIZADOS: MSE = 18.7641
